In [1]:
!pip -q install langchain[llms] openai pypdf tiktoken PyPDF2 transformers accelerate bitsandbytes torch sentencepiece


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: C:\Users\myfir\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:

from langchain.text_splitter import TokenTextSplitter
import os
import textwrap
import re

In [3]:

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are an AI Assistant, Carefully analyze this part of document. Summarize the Folling Text for me:"""


In [4]:

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"  # Insert citation here

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")


def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')

In [5]:


def clean_extra_whitespace(text):
    """
    Removes extra whitespaces from the text.
    """
    text = re.sub(r'\s+', ' ', text).strip()  # Replace multiple spaces with a single space
    return text

def clean_non_ascii_chars(text):
    """
    Removes non-ASCII characters from the text.
    """
    return ''.join(char for char in text if ord(char) < 128)

def group_broken_paragraphs(text):
    """
    Groups broken paragraphs. Assumes that a paragraph is broken if it ends with a hyphen followed immediately by a word.
    """
    return re.sub(r'-\s*([^ \n]+)', r'\1', text)

def clean_text(text):
    """
    Combined function to clean the text.
    """
    text = clean_extra_whitespace(text)
    text = clean_non_ascii_chars(text)
    text = group_broken_paragraphs(text)
    return text


In [ ]:
default_template = """
              You are Helpful Assistant, Carefully analyze the given chunk of document.
              Write a summary of the following text delimited by triple backticks.
              Dont Loose Important Information and Provide All Relevant, Accurate Information.
              Return your response which covers all key points of the chunk.
              ```{text}```
              SUMMARY:
              """


simple_system_prompt="You are Helpful Assistant, Master at Summarizing Docs, Carefully analyze the provided chunk of document."
simple_instruction="""Write the best summary of the following chunk Given Below :
              -------------------------------
              ```{text}```
              -------------------------------

              Write a concise Summary which is As Accurate as Possible and Covers all aspects of the this Document Chunk .
              Include Relevant Information like Document Identification, Authorship and Context, Content Summary.
              ONLY ANSWER IN BULLET POINTS:"""

simple_template = get_prompt(simple_instruction, simple_system_prompt)
#print(simple_template)


prompt_template = """Extract all the Information from this section of the document given below:
    Extract Key Parts,Important Facts,Relevant Details,Only Provide Accurate Information. Include the data only from below chunk.
    ```{text}```
    Summary should be concise, well formatted, covers aspects of the this Chunk, can have Bullet Points and completely accurate.
    """

List_system_prompt="You are Helpful Assistant, Master at Information Extraction and Summarization of Docs, I will Provide You with Multiple Chunks of One Document, Your Job is to Extract all Information and List Them Down."
List_Refine_template = """Cover all aspects in detail and be completely accurate. Extract all the Information from the Chunks given below: 
    -----------------

    ```{text}```
    
    ----------------

    Give the Summary having below section.
    Context : (Summary in Bullet Points).  
    Dont Loose Details and Keep it coherent and concise.
    """
# Mention Relevant Information if available (Case / Document Details, Citations, Dates, Judges,other relevant details)
Refined_Final_template = get_prompt(List_Refine_template, List_system_prompt)
print(Refined_Final_template)

In [ ]:
First_system_prompt="You are Helpful Assistant, Master at Information Extraction of Docs, I will Provide You with Chunk of a Document, Your Job is to Extract all Important Information and List Them Down."
First_Refine_template = """Extract all Important Details,Only Provide Accurate Information.
    Include the data only from below chunk.
    ---------------------------

    ```{text}```

    ---------------------------
    Include Below Information if available:
    A
    B
    C
    D
    
    Be Concise and Dont Loose any Details. Cover all aspects and be completely accurate.
    Extract all Information.
    """
First_Final_template = get_prompt(First_Refine_template, First_system_prompt)
print(First_Final_template)

In [9]:
import requests
import json
import time

# Replace this with your actual pod ID
POD_ID = "RUNPOD_ID"
BASE_URL = f"https://{POD_ID}-8080.proxy.runpod.net"

def generate_text(
    inputs, 
    best_of=1,
    decoder_input_details=True,
    details=True,
    do_sample=True,
    max_new_tokens=1024,
    repetition_penalty=1.1,
    return_full_text=False,
    temperature=0.01,
    top_k=30,
    truncate=None,
    stream=False
):
    payload = {
        "inputs": inputs,
        "parameters": {
            "best_of": best_of,
            "decoder_input_details": decoder_input_details,
            "details": details,
            "do_sample": do_sample,
            "max_new_tokens": max_new_tokens,
            "repetition_penalty": repetition_penalty,
            "return_full_text": return_full_text,
            "temperature": temperature,
            "top_k": top_k,
            "truncate": truncate
        },
        "stream": stream
    }

    # Attempt the request with retry mechanism
    for attempt in range(2):  # 2 attempts in total
        try:
            response = requests.post(
                BASE_URL,
                headers={"Content-Type": "application/json"},
                data=json.dumps(payload),
                timeout=300  # 5 minutes in seconds
            )
            
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Attempt {attempt + 1}: Unexpected status code {response.status_code}: {response.text}")
                if attempt == 0:  # if this is the first attempt
                    time.sleep(2)  # wait for 2 seconds before retrying
                else:
                    return {"error": "OOPS"}

        except (requests.RequestException, requests.Timeout) as e:
            print(f"Attempt {attempt + 1}: Request failed with exception: {e}")
            if attempt == 0:  # if this is the first attempt
                time.sleep(2)  # wait for 2 seconds before retrying
            else:
                return {"error": "OOPS"}



In [10]:
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
import torch

def process_directory_with_stuff_chain(root_dir):

    for subdir, _, files in os.walk(root_dir):
        print(f"Processing directory: {subdir}")
        for file in files:
            if file.endswith('.txt') and not file.endswith('_cleaned.txt') and not file.endswith('_SLS.txt'):
                file_path = os.path.join(subdir, file)
                print(f"Processing file: {file_path}")

                with open(file_path, 'r', encoding='utf-8') as infile:
                    content = infile.read()
                if not content:
                    print(f"File {file_path} is empty!")
                    continue
                all_summaries = []
                common_summary = ""
                cleaned_content = clean_text(content)

                Details_splitter = TokenTextSplitter(chunk_size=1250, chunk_overlap=250)
                details_texts = Details_splitter.split_text(cleaned_content)
                first_doc = Document(page_content=details_texts[0])
                First_Prompt = First_Final_template.replace("```{text}```", first_doc.page_content)
                exracted_info = generate_text(First_Prompt)
                Initial_exracted_info = exracted_info[0]['generated_text']
                print("Initial Extraction --------- "+ Initial_exracted_info[:200])
                all_summaries.append(Initial_exracted_info)
                

                text_splitter = TokenTextSplitter(chunk_size=1250, chunk_overlap=300)
                texts = text_splitter.split_text(cleaned_content)
                docs = [Document(page_content=t) for t in texts]
               
                for i, doc in enumerate(docs):
                    print(f"Processing chunk {i + 1} of {len(docs)}")
                    
                    try:
                        Final_prompt = Refined_Final_template.replace("```{text}```", doc.page_content)
                        response_text = generate_text(Final_prompt)
                        summarized_doc = response_text[0]['generated_text']
                        print("First few chars of summary:", summarized_doc[:100])
                        if not summarized_doc:
                            print(f"No summary generated for chunk {i + 1}")
                            continue
                        all_summaries.append(summarized_doc)
                        common_summary += " " + summarized_doc
                        torch.cuda.empty_cache()
                    except Exception as e:
                        print(f"Error processing chunk {i + 1}: {e}")
                        continue

                if not all_summaries:
                    print(f"No summaries generated for file {file_path}")
                    continue
                
                summary_file_path = os.path.join(subdir, file.split('.txt')[0] + '_SLS.txt')
                with open(summary_file_path, 'w', encoding='utf-8') as outfile:
                    outfile.write('\n'.join(all_summaries))
                    print(f"Summaries saved to: {summary_file_path}")


START MASS SUMMARIZATION

In [ ]:
# Define the base directory path
base_path = r'C:\Users\myfir\Downloads\UNC\united_nation\CED\UNCED'
zip_path = base_path + ".zip"

# Call the processing function
#process_directory_summarize("/content/test", llm)
process_directory_with_stuff_chain(base_path)
#Passed

In [ ]:

import shutil
# Create a zip file
shutil.make_archive(base_path, 'zip', base_path)

# Download the zip file using Google Colab's utility
files.download(zip_path)